In [31]:
import pandas as pd

In [32]:
import os

In [36]:
files = [f for f in os.listdir("../Generating Adversarial Samples/Data/") if not f.startswith('.')]
files

['rotten-tomatoes_pwws_distilbert.csv',
 'imdb_pwws_cnn.csv',
 'ag-news_bae_distilbert.csv',
 'ag-news_pwws_distilbert.csv',
 'imdb_bae_distilbert.csv',
 'ag-news_pwws_cnn.csv',
 'ag-news_pwws_lstm.csv',
 'ag-news_kuleshov_distilbert.csv',
 'ag-news_pwws_bert.csv',
 'imdb_pwws_distilbert.csv',
 'imdb_alzantot_distilbert.csv',
 'rotten-tomatoes_alzantot_distilbert.csv',
 'yelp-polarity_textfooler_bert.csv',
 'imdb_kuleshov_distilbert.csv',
 'rotten-tomatoes_bae_distilbert.csv',
 'ag-news_alzantot_distilbert.csv',
 'imdb_pwws_bert.csv',
 'imdb_pwws_lstm.csv',
 'yelp-polarity_pwws_bert.csv',
 'ag-news_textfooler_distilbert.csv',
 'imdb_textfooler_distilbert.csv']

In [22]:
df = pd.read_csv(f'../Generating Adversarial Samples/Data/imdb_pwws_lstm.csv', index_col=0)

In [23]:
df.head()

,original_text,adversarial_text,original_class,original_confidence,adversarial_class,adversarial_confidence,attack,replace_dict,replace_num
0,I'm surprised that the comparison hasn't been ...,I'm surprised that the comparison hasn't been ...,Negative,(68%),Positive,(71%),pwws,{'Awful': 'tremendous'},1
1,"The last good Ernest movie, and the best at th...","The last good Ernest movie, and the best at th...",Positive,(99%),Negative,(62%),pwws,"{'last': 'endure', 'line': 'tune', 'lines': 'c...",5
2,I just found the entire 3 DVD set at Wal-Mart ...,I just found the entire 3 DVD set at Wal-Mart ...,Positive,(84%),Negative,(72%),pwws,{'dependent': 'drug-addicted'},1
3,This is a absolutely masterful stroke of geniu...,This is a absolutely masterful stroke of geniu...,Positive,(99%),Negative,(53%),pwws,"{'examines': 'canvass', 'finest': 'okay', 'Mar...",3
4,"I consider Saboteur as Hitchcock's first ""Amer...","I consider Saboteur as Hitchcock's first ""Amer...",Positive,(95%),Negative,(82%),pwws,"{'brilliant': 'brainy', 'Best': 'respectable'}",2


In [43]:
def tokenize(text):
    text = text.lower()
    return nltk.word_tokenize(text)

import ast

def dict_to_tuples(row):
    output = []
    replace_dict = ast.literal_eval(row['replace_dict'])
    
    try:
    
        for k in replace_dict:
            indexes = (i for i, e in enumerate(row['clean']) if e == k)
            found = 0

            for i in indexes:
                if row['perturbed'][i] == replace_dict[k]:
                    output.append((k, replace_dict[k].lower(), i))
                    found = 1
                if found == 1:
                    break

            if found==0:
                output.append((k.lower(), replace_dict[k].lower(), 0))
    
    except:
        
        return []
                
    return output

In [38]:
for file in files:
    df = pd.read_csv(f'../Generating Adversarial Samples/Data/{file}',
                        index_col=0)
    print(f"{file}: {len(df)}")

rotten-tomatoes_pwws_distilbert.csv: 706
imdb_pwws_cnn.csv: 809
ag-news_bae_distilbert.csv: 316
ag-news_pwws_distilbert.csv: 1509
imdb_bae_distilbert.csv: 824
ag-news_pwws_cnn.csv: 630
ag-news_pwws_lstm.csv: 587
ag-news_kuleshov_distilbert.csv: 795
ag-news_pwws_bert.csv: 424
imdb_pwws_distilbert.csv: 28492
imdb_alzantot_distilbert.csv: 734
rotten-tomatoes_alzantot_distilbert.csv: 495
yelp-polarity_textfooler_bert.csv: 921
imdb_kuleshov_distilbert.csv: 101
rotten-tomatoes_bae_distilbert.csv: 484
ag-news_alzantot_distilbert.csv: 223
imdb_pwws_bert.csv: 712
imdb_pwws_lstm.csv: 819
yelp-polarity_pwws_bert.csv: 459
ag-news_textfooler_distilbert.csv: 648
imdb_textfooler_distilbert.csv: 897


In [44]:
import pickle
import nltk

existing = list(os.listdir(f'./Data FGWS/'))

for file in files:
    if file.replace('.csv', '') in existing:
        continue
    
    print(file)
    
    setup = file.split('_')
    dataset = setup[0]
    attack = setup[1]
    model = setup[2].replace('.csv', '')

    df = pd.read_csv(f'../Generating Adversarial Samples/Data/{file}',
                index_col=0)

    if 'replace_dict' not in list(df.columns):
        df['replace_dict'] = ["{}" for i in range(len(df))]

    df[['original_class', 'adversarial_class']] = df[['original_class', 'adversarial_class']].replace('Negative', 0)
    df[['original_class', 'adversarial_class']] = df[['original_class', 'adversarial_class']].replace('Positive', 1)
    df[['original_class', 'adversarial_class']] = df[['original_class', 'adversarial_class']].replace('World', 0)
    df[['original_class', 'adversarial_class']] = df[['original_class', 'adversarial_class']].replace('Sports', 1)
    df[['original_class', 'adversarial_class']] = df[['original_class', 'adversarial_class']].replace('Business', 2)
    df[['original_class', 'adversarial_class']] = df[['original_class', 'adversarial_class']].replace('Sci/tech', 3)

    df['original_class_predicted'] = df['original_class']
    df['adversarial_class_predicted'] = df['adversarial_class']

    df_short = df[['original_text', 'adversarial_text', 'original_class', 'original_class_predicted', 'adversarial_class_predicted', 'replace_dict']]

    df_short = df_short.rename(columns={
                                "original_text": 'clean',
                                'adversarial_text': 'perturbed',
                                'original_class': 'label',
                                'original_class_predicted': 'clean_pred',
                                'adversarial_class_predicted': 'perturbed_pred'
                                })

    df_short['clean'] = df_short['clean'].apply(tokenize)
    df_short['perturbed'] = df_short['perturbed'].apply(tokenize)

    df_short['perturbed_idxs'] = df_short.apply(lambda row: dict_to_tuples(row), axis = 1)

    df_short = df_short.drop(columns=['replace_dict'])

    test_texts = list(df_short['clean'].values)
    test_pols = list(df_short['label'].values)

    test_dict = df_short.to_dict('records')

    os.mkdir(f'./Data FGWS/{dataset}_{attack}_{model}/')

    with open(f'./Data FGWS/{dataset}_{attack}_{model}/test_texts.pkl', "wb") as handle:
        pickle.dump(test_texts, handle)

    with open(f'./Data FGWS/{dataset}_{attack}_{model}/test_pols.pkl', "wb") as handle:
        pickle.dump(test_pols, handle)

    with open(f'./Data FGWS/{dataset}_{attack}_{model}/adv_examples.pkl', "wb") as handle:
        pickle.dump(test_dict, handle)

imdb_bae_distilbert.csv
rotten-tomatoes_bae_distilbert.csv
